# Trend Lines and Channels Detection

In [4]:
import ccxt
import pandas as pd

# exchange = ccxt.binance()
# exchange.load_markets()

# '''symbols variabl contains all the symbols of the exchange '''
# symbols = exchange.symbols

# # ticker = 'BTC-USDT'
# # timeframe ='4h'
# def ccxt_data(symbol='HNT/USDT', timeframe ='4h', limit=9000):

#     """ use global to make a variable accesible by oher functions
#     and always declare it before using it  """
#     global df

#     '''if you dont do timeframe=timeframe it will not work 
#     but in symbol it works !!'''
#     # fetch returns a list
#     bars = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)

#     df = pd.DataFrame(bars[:], columns=['time', 'open', 'high', 'low', 'close', 'volume'])
#     # Time is in unix format
#     df['time'] = pd.to_datetime(df['time'], unit='ms')
#     df.set_index('time', inplace=True)
    
#     return df

df = pd.read_csv('data/2022-01-28_11/4h/ATOM_USDT_4h_11111.csv', index_col='Time')



In [5]:

#Check if NA values are in data
df=df[df['Volume']!=0]
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df.head(10)

,Open,High,Low,Close,Volume
0,15.147,15.587,14.740,15.560,422179.643
1,15.560,15.919,15.310,15.489,501349.852
2,15.505,15.550,14.900,14.957,394829.093
3,14.961,15.385,14.874,15.265,281546.584
4,15.262,15.482,14.891,15.162,375634.342
5,15.153,15.480,14.934,15.441,288387.990
6,15.442,15.955,15.424,15.910,365037.071
7,15.912,16.154,15.674,15.725,499864.405
8,15.724,16.279,15.692,16.245,443625.003
9,16.235,16.520,15.961,16.484,471308.993


In [6]:
dfpl = df[:]
import plotly.graph_objects as go
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.show()

In [7]:
import numpy as np
from matplotlib import pyplot
backcandles= 50
wind = 5

candleid = 880

maxim = np.array([])
minim = np.array([])
xxmin = np.array([])
xxmax = np.array([])
for i in range(candleid-backcandles, candleid+1, wind):
    minim = np.append(minim, df.Low.iloc[i:i+wind].min())
    xxmin = np.append(xxmin, df.Low.iloc[i:i+wind].idxmin())
for i in range(candleid-backcandles, candleid+1, wind):
    maxim = np.append(maxim, df.High.loc[i:i+wind].max())
    xxmax = np.append(xxmax, df.High.iloc[i:i+wind].idxmax())
slmin, intercmin = np.polyfit(xxmin, minim,1)
slmax, intercmax = np.polyfit(xxmax, maxim,1)

dfpl = df[candleid-backcandles:candleid+backcandles]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])
fig.add_trace(go.Scatter(x=xxmin, y=slmin*xxmin + intercmin, mode='lines', name='min slope'))
fig.add_trace(go.Scatter(x=xxmax, y=slmax*xxmax + intercmax, mode='lines', name='max slope'))

# Fitting intercepts to meet highest or lowest candle point in time slice

In [8]:
dfpl = df[candleid-wind-backcandles:candleid+backcandles]
import plotly.graph_objects as go
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

#-------------------------------------------------------------------------
# Fitting intercepts to meet highest or lowest candle point in time slice
adjintercmin = df.Low.loc[candleid-backcandles:candleid].min() - slmin*df.Low.iloc[candleid-backcandles:candleid].idxmin()
adjintercmax = df.High.loc[candleid-backcandles:candleid].max() - slmax*df.High.iloc[candleid-backcandles:candleid].idxmax()
fig.add_trace(go.Scatter(x=xxmin, y=slmin*xxmin + adjintercmin, mode='lines', name='min slope'))
fig.add_trace(go.Scatter(x=xxmax, y=slmax*xxmax + adjintercmax, mode='lines', name='max slope'))
fig.show()

# Fitting intercepts to wrap price candles

In [11]:
dfpl = df[candleid-wind-backcandles:candleid+backcandles]
import plotly.graph_objects as go
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

#-------------------------------------------------------------------------
# Fitting intercepts to wrap price candles
adjintercmax = (df.High.iloc[xxmax] - slmax*xxmax).max()
adjintercmin = (df.Low.iloc[xxmin] - slmin*xxmin).min()
fig.add_trace(go.Scatter(x=xxmin, y=slmin*xxmin + adjintercmin, mode='lines', name='min slope'))
fig.add_trace(go.Scatter(x=xxmax, y=slmax*xxmax + adjintercmax, mode='lines', name='max slope'))
fig.show()

# backcandles time window more dynamic

In [15]:
import numpy as np
from matplotlib import pyplot
backcandles= 40 # 6*8
brange = 10 # backcandles//4 #should be less than backcandles
wind = 6

candleid = 500

optbackcandles= backcandles
sldiff = 999

for r1 in range(backcandles-brange, backcandles+brange):
    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])
    for i in range(candleid-r1, candleid+1, wind):
        minim = np.append(minim, df.Low.iloc[i:i+wind].min())
        xxmin = np.append(xxmin, df.Low.iloc[i:i+wind].idxmin())
    for i in range(candleid-r1, candleid+1, wind):
        maxim = np.append(maxim, df.High.loc[i:i+wind].max())
        xxmax = np.append(xxmax, df.High.iloc[i:i+wind].idxmax())
    slmin, intercmin = np.polyfit(xxmin, minim,1)
    slmax, intercmax = np.polyfit(xxmax, maxim,1)
    
    if(abs(slmin-slmax)<sldiff):
        sldiff = abs(slmin-slmax)
        optbackcandles=r1
        slminopt = slmin
        slmaxopt = slmax
        intercminopt = intercmin
        intercmaxopt = intercmax
        maximopt = maxim.copy()
        minimopt = minim.copy()
        xxminopt = xxmin.copy()
        xxmaxopt = xxmax.copy()

        print(optbackcandles)
dfpl = df[candleid-wind-optbackcandles-backcandles:candleid+optbackcandles]  
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

adjintercmax = (df.High.iloc[xxmaxopt] - slmaxopt*xxmaxopt).max()
adjintercmin = (df.Low.iloc[xxminopt] - slminopt*xxminopt).min()
fig.add_trace(go.Scatter(x=xxminopt, y=slminopt*xxminopt + adjintercmin, mode='lines', name='min slope'))
fig.add_trace(go.Scatter(x=xxmaxopt, y=slmaxopt*xxmaxopt + adjintercmax, mode='lines', name='max slope'))
fig.show()

30
31
32
33
38


In [13]:
import numpy as np
from matplotlib import pyplot
backcandles= 88
brange = 50 #should be less than backcandles
wind = 5

candleid = 700

optbackcandles= backcandles
sldiff = 10
sldist = 55
for r1 in range(backcandles-brange, backcandles+brange):
    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])
    
    for i in range(candleid-r1, candleid+1, wind):
        minim = np.append(minim, df.Low.iloc[i:i+wind].min())
        xxmin = np.append(xxmin, df.Low.iloc[i:i+wind].idxmin())
    for i in range(candleid-r1, candleid+1, wind):
        maxim = np.append(maxim, df.High.loc[i:i+wind].max())
        xxmax = np.append(xxmax, df.High.iloc[i:i+wind].idxmax())
    slmin, intercmin = np.polyfit(xxmin, minim,1)
    slmax, intercmax = np.polyfit(xxmax, maxim,1)
    
    dist = (slmax*candleid + intercmax)-(slmin*candleid + intercmin)
    if(dist<sldist): #abs(slmin-slmax)<sldiff and
        #sldiff = abs(slmin-slmax)
        sldist = dist
        optbackcandles=r1
        slminopt = slmin
        slmaxopt = slmax
        intercminopt = intercmin
        intercmaxopt = intercmax
        maximopt = maxim.copy()
        minimopt = minim.copy()
        xxminopt = xxmin.copy()
        xxmaxopt = xxmax.copy()

        
print(optbackcandles)
dfpl = df[candleid-wind-optbackcandles-backcandles:candleid+optbackcandles]  
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

adjintercmax = (df.High.iloc[xxmaxopt] - slmaxopt*xxmaxopt).max()
adjintercmin = (df.Low.iloc[xxminopt] - slminopt*xxminopt).min()
fig.add_trace(go.Scatter(x=xxminopt, y=slminopt*xxminopt + adjintercmin, mode='lines', name='min slope'))
fig.add_trace(go.Scatter(x=xxmaxopt, y=slmaxopt*xxmaxopt + adjintercmax, mode='lines', name='max slope'))
fig.show()

40
